# Exercise Solutions

---

> Author: <font color='#f78c40'>Samuel Farrens</font>    
> Year: 2018  
> Email: [samuel.farrens@cea.fr](mailto:samuel.farrens@cea.fr)  
> Website: <a href="https://sfarrens.github.io" target="_blank">https://sfarrens.github.io</a>

---


## Contents

1. [Introduction to Inverse Problems](#Introduction-to-Inverse-Problems)
 * [Linear Regression Exercise](#Linear-Regression-Exercise)
 * [Gradient Descent Exercise](#Gradient-Descent-Exercise)
1. [Introduction to Sparsity I](#Introduction-to-Sparsity-I)
 * [Denoising Example](#Denoising-Example)
1. [Introduction to Sparsity II: Compressed Sensing](#Introduction-to-Sparsity-II:-Compressed-Sensing)
 * [1D Example](#1D-Example)
 * [2D Exercise](#2D-Exercise)
1. [Introduction to Sparsity III: Deconvolution](#Introduction-to-Sparsity-II:-Deconvolution)
 * [Deconvolution Exercise](#Deconvolution-Exercise)

---
## Introduction to Inverse Problems

### <font color='blue'>Linear Regression Exercise</font>

```Python
# Define the matrix operator X here:
H_k = np.array([np.ones(x_k.size), x_k, x_k ** 2, x_k ** 3]).T

# Calculate the model parameters A here:
a_k = normal_eq(H_k, y_k)
```

### <font color='blue'>Gradient Descent Exercise</font>

* The gradient and cost function are the same as for the straight line example. 
* It takes around 20,000 iterations with $\gamma = 0.03$.

```Python
# Solution
a_gd, cost_gd = grad_desc(y_k, [0, 0, 0, 0], H_k, grad, cost_func, gamma=0.03, n_iter=20000)
```

---

## Introduction to Sparsity I

### <font color='blue'>Denoising Example</font>

```Python
# Define the sparse representations of the signals.
x_sparse = fft(x)
y_sparse = fft(y)

# Determine the value of lambda.
lamdba_val = 3 * sigma_mad(y_sparse) 

# Perform thresholding.
x_rec = ifft(hard_thresh(y_sparse, lamda_val))
```

---

## Introduction to Sparsity II: Compressed Sensing

### <font color='blue'>1D Example</font>

```Python
# generate signal x
x = nfft(alpha)

# set the number of non-zero coefficients retained
n_subsamples = n_samples // 4

# generate the mask
mask = np.random.permutation(np.hstack((np.ones(n_subsamples), np.zeros(n_samples - n_subsamples))))

# generate the subsampled observation y
y = mask_op(x, mask)

# Function to calculate the gradient
def grad(y, alpha_rec, mask):
    
    return infft(upsample(mask_op(nfft(alpha_rec), mask) - y, mask))

#Test NMSE after 10 iterations
nmse10 = nmse(alpha, forwardBackward(y, np.ones(n_samples), mask, grad, 0.01, 10, 1.0))

# Test NMSE after 200 iterations
nmse200 = nmse(alpha, alpha_rec)
```

NMSE after 10 iterations: 0.6204572754108413  
NMSE after 200 iterations: 0.0007156246947551836

### <font color='blue'>2D Exercise</font>

The gradient and optimisation problem are the same as for the 1D example.

```Python
sample_percentage = mask2d[mask2d == 1].size / float(mask2d.size) * 100)
```
The percentage of samples used is: 5.0%

```Python
# Recover the sparse singal alpha
alpha_rec2d, cost2d = forwardBackward(y2d, first_guess=np.ones(mask2d.size), mask=mask2d, grad=grad, 
                                      lambda_val=0.05, n_iter=600, return_cost=True)

# Convert back to signal domain
x_rec2d = nfft(alpha_rec2d)

# Calculate NMSE
nmse_val = nmse(np.load('data/cs_true_data.npy'), x_rec2d)
```
NMSE = 0.0017443061956

It takes roughly 500 iterations to converge.

---

## Introduction to Sparsity III: Deconvolution|

### <font color='blue'>Deconvolution Exercise</font>

```Python
x_analytical = fftdeconvolve(y, psf)
```

Nope, doesn't work.

```Python
# The gradient
def grad(y, alpha_rec, psf):
    
    return ifft(Ht(H(fft(alpha_rec), psf) - y, psf))

# The cost function
def cost_func(y, alpha_rec, psf, lambda_):
    
    return (0.5 * np.linalg.norm(y - H(fft(alpha_rec), psf)) ** 2 + lambda_ * l1_norm(alpha_rec))

# Deconvolve the image
alpha_rec, cost = forwardBackward(y, np.ones(y.shape), psf, grad, lambda_val=0.003, n_iter=400, 
                                  return_cost=True)

# Convert back to signal domain
x_rec = fft(alpha_rec)

# Calculate NMSE
nmse_val = nmse(np.load('data/cs_true_data.npy'), x_rec.flatten())
```

NMSE = 0.00523458123461

It takes roughly 350 iterations to converge.

---

## Introduction to Wavelets

### <font color='blue'>Starlet Transform</font>

```Python
# Reconstruct the image of Venus.
venus = np.sum(venus_scales, axis=0)
```